In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys
sys.path.append('/work/qdiff/mo_utils')
sys.executable


'/home/nadavg/anaconda3/envs/qdiff/bin/python'

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [4]:
#from mo_utils.utils.torch_utils import torch_to_pt,create_torch_script
import netron
from pathlib import Path

In [5]:
from qdiff import (
    QuantModel, QuantModule, BaseQuantBlock, 
    block_reconstruction, layer_reconstruction,
)
from qdiff.utils import resume_cali_model, get_train_samples


In [6]:
import torch
from omegaconf import OmegaConf
from txt2img import load_model_from_config
from ldm.models.diffusion.plms import PLMSSampler

/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

In [7]:
#cali_ckpt = '/fastdata/users/nadavg/sd/qdiff/sd_w4a8_ckpt.pth'
#cali_ckpt,nbit,symmetric   = '/fastdata/users/nadavg/sd/qdiff/sd_w8a8_ckpt.pth' ,8,False
#w8bit_nosym,nbit,symmetric = '/fastdata/users/nadavg/sd/qdiff/output_quantization/2025-01-22-16-39-34/wc_ckpt.pth',8,False
#w8bit_sym = "/fastdata/users/nadavg/sd/qdiff/output_quantization/2025-01-22-16-40-17/wc_ckpt.pth"
#w8bit_sym,nbit,symmetric   = '/home/nadavg/q-diffusion/output_quantization2/2025-01-24-00-49-55/wc_ckpt.pth',8,True
#w8bit_sym,nbit,symmetric = '/home/nadavg/q-diffusion/output_quantization3/2025-01-26-18-44-03/wc_ckpt.pth',8,True
w8bit_sym,nbit,symmetric = '/fastdata/users/nadavg/sd/qdiff/output_quantization/2025-01-26-18-44-03/ckpt.pth',8,True

In [8]:
#ckpt_full = torch.load(cali_ckpt, map_location='cpu')
#ckpt_weight_only = torch.load(weight_ckpt, map_location='cpu')

In [9]:
# import garbage collector
from gc import collect as gc
#from torch import g
gc()

109

In [10]:
config = OmegaConf.load(f'{Path.home()}/q-diffusion/configs/stable-diffusion/v1-inference.yaml')
model = load_model_from_config(config, "/fastdata/users/nadavg/sd/qdiff/sd-v1-4.ckpt")
device = torch.device("cuda")
model = model.to(device)
sampler = PLMSSampler(model)

/home/nadavg/q-diffusion/scripts/txt2img.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_sd = torch.load(ckpt, map_location="cpu")


LatentDiffusion: Running in eps-prediction mode


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.1.layer_norm1.weight', 'vision_model.encoder.layers.11.mlp.fc1.weight', 'vision_model.encoder.layers.15.layer_norm2.bias', 'vision_model.encoder.layers.6.layer_norm2.weight', 'vision_model.encoder.layers.17.self_attn.k_proj.weight', 'vision_model.encoder.layers.8.mlp.fc1.bias', 'vision_model.encoder.layers.23.mlp.fc1.bias', 'vision_model.encoder.layers.3.mlp.fc1.weight', 'vision_model.encoder.layers.11.mlp.fc1.bias', 'vision_model.encoder.layers.8.layer_norm2.weight', 'vision_model.encoder.layers.23.self_attn.q_proj.weight', 'vision_model.encoder.layers.22.self_attn.k_proj.bias', 'vision_model.encoder.layers.14.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.mlp.fc2.weight', 'vision_model.encoder.layers.23.mlp.fc1.weight', 'vision_model.encoder.layers.10.mlp.fc1.bias', 'vision_model.encoder.layers.22.self_attn.out_proj.weight', 'text

In [11]:
setattr(sampler.model.model.diffusion_model, "split", True)

In [13]:
#wq_params = {'n_bits': 4, 'channel_wise': True, 'scale_method': 'mse'}
wq_params = {'n_bits': nbit, 'channel_wise': True, 'scale_method': 'mse', 'symmetric': symmetric}

aq_params = {'n_bits': 8, 'channel_wise': False, 'scale_method': 'mse', 'leaf_param':  True}
wq_params['scale_method'] = 'max'
aq_params['scale_method'] = 'max'

In [14]:
from qdiff.quant_layer import QuantModule,QuantOp

In [14]:
df=sampler.model.model.diffusion_model

In [15]:
rsb = list(df.named_children())[1][1][1][0]#.input_blocks
rsb.in_layers[1]

SiLU()

In [16]:
qop=QuantOp(rsb.in_layers[0],aq_params)

In [17]:
isinstance(qop,torch.nn.Module)

True

In [15]:
qnn = QuantModel(model=sampler.model.model.diffusion_model,
                weight_quant_params=wq_params, act_quant_params=aq_params,
                act_quant_mode="qdiff", sm_abit=16)

In [16]:
qnn.specials

{ldm.modules.diffusionmodules.openaimodel.ResBlock: qdiff.quant_block.QuantResBlock,
 ldm.modules.attention.BasicTransformerBlock: qdiff.quant_block.QuantBasicTransformerBlock,
 ddim.models.diffusion.ResnetBlock: qdiff.quant_block.QuantResnetBlock,
 ddim.models.diffusion.AttnBlock: qdiff.quant_block.QuantAttnBlock,
 ldm.modules.diffusionmodules.openaimodel.QKMatMul: qdiff.quant_block.QuantQKMatMul,
 ldm.modules.diffusionmodules.openaimodel.SMVMatMul: qdiff.quant_block.QuantSMVMatMul}

In [17]:
qnn.cuda()
qnn.eval()
qnn.set_grad_ckpt(False)

In [18]:
w8bit_sym

'/fastdata/users/nadavg/sd/qdiff/output_quantization/2025-01-26-18-44-03/ckpt.pth'

In [ ]:
cali_data = (torch.randn(1, 4, 64, 64), torch.randint(0, 1000, (1,)), torch.randn(1, 77, 768))
resume_cali_model(qnn, w8bit_sym, cali_data, True, "qdiff", cond=True)


In [39]:
cali_data = (torch.randn(1, 4, 64, 64), torch.randint(0, 1000, (1,)), torch.randn(1, 77, 768))
qnn.set_quant_state(True, True)

In [40]:
cali_xs, cali_ts, cali_cs = cali_data
_ = qnn(cali_xs[:1].cuda(), cali_ts[:1].cuda(), cali_cs[:1].cuda())

In [41]:
type(sampler.model.model.diffusion_model)
sampler.model.model.diffusion_model.split

True

In [26]:
# add full name to each module
for name, module in qnn.named_modules():
    module.full_name = name

In [44]:
from qdiff.quant_layer import UniformAffineQuantizer
for name, module in qnn.named_modules():
    if isinstance(module, QuantModule):
        if module.split:
            print('split:' ,module.full_name)# = name
            break
        #else:
        #    print('no split:' ,module.full_name)

split: model.output_blocks.0.0.skip_connection


In [45]:
module

QuantModule(
  2560, 1280, kernel_size=(1, 1), stride=(1, 1)
  (weight_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=True, channel_wise=True, leaf_param=False)
  (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
  (activation_function): StraightThrough()
  (weight_quantizer_0): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=True, channel_wise=True, leaf_param=False)
  (act_quantizer_0): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
)

In [ ]:
from qdiff.quant_layer import UniformAffineQuantizer
for name, module in qnn.named_modules():
    if isinstance(module, UniformAffineQuantizer):
        if module.leaf_param:
            print('leaf:' ,module.full_name)# = name
        else:
            print('no leaf:' ,module.full_name)

no leaf: model.time_embed.0.weight_quantizer
leaf: model.time_embed.0.act_quantizer
leaf: model.time_embed.1.act_quantizer
no leaf: model.time_embed.2.weight_quantizer
leaf: model.time_embed.2.act_quantizer
no leaf: model.input_blocks.0.0.weight_quantizer
leaf: model.input_blocks.0.0.act_quantizer
leaf: model.input_blocks.1.0.act_quantizer
leaf: model.input_blocks.1.0.in_layers.0.act_quantizer
leaf: model.input_blocks.1.0.in_layers.1.act_quantizer
no leaf: model.input_blocks.1.0.in_layers.2.weight_quantizer
leaf: model.input_blocks.1.0.in_layers.2.act_quantizer
leaf: model.input_blocks.1.0.emb_layers.0.act_quantizer
no leaf: model.input_blocks.1.0.emb_layers.1.weight_quantizer
leaf: model.input_blocks.1.0.emb_layers.1.act_quantizer
leaf: model.input_blocks.1.0.out_layers.0.act_quantizer
leaf: model.input_blocks.1.0.out_layers.1.act_quantizer
no leaf: model.input_blocks.1.0.out_layers.3.weight_quantizer
leaf: model.input_blocks.1.0.out_layers.3.act_quantizer
no leaf: model.input_blocks.

In [25]:
bool(qnn.model.input_blocks[1][0].skip_connection)

True

In [26]:
for module in qnn.modules():
    if getattr(module,'skip_connection',False):
        print(module.full_name)
        #print(module)

model.input_blocks.1.0
model.input_blocks.2.0
model.input_blocks.4.0
model.input_blocks.5.0
model.input_blocks.7.0
model.input_blocks.8.0
model.input_blocks.10.0
model.input_blocks.11.0
model.middle_block.0
model.middle_block.2
model.output_blocks.0.0
model.output_blocks.1.0
model.output_blocks.2.0
model.output_blocks.3.0
model.output_blocks.4.0
model.output_blocks.5.0
model.output_blocks.6.0
model.output_blocks.7.0
model.output_blocks.8.0
model.output_blocks.9.0
model.output_blocks.10.0
model.output_blocks.11.0


In [27]:
len(list(qnn.model.named_children())[1])

2

In [28]:
rsb = list(qnn.model.named_children())[1][1][1][0]#.input_blocks

In [29]:
type(rsb)

qdiff.quant_block.QuantResBlock

In [30]:
type(rsb.in_layers[1])#.num_groups, rsb.in_layers[0].num_channels

qdiff.quant_layer.QuantOp

In [31]:
rsb.in_layers[1]

QuantOp(
  (fwd_func): SiLU()
  (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
  (activation_function): StraightThrough()
)

In [32]:
import torch.nn as nn
from ldm.modules.diffusionmodules.util import GroupNorm32

In [33]:
isinstance(rsb.in_layers[0],GroupNorm32)

False

In [55]:
cali_data = (torch.randn(1, 4, 64, 64), torch.randint(0, 1000, (1,)), torch.randn(1, 77, 768))
#resume_cali_model(qnn, cali_ckpt, cali_data, False, "qdiff", cond=True)
resume_cali_model(qnn, weight_ckpt, cali_data, False, "qdiff", cond=True)

Loading quantized model checkpoint


/home/nadavg/q-diffusion/qdiff/utils.py:384: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


Initializing weight quantization parameters


RuntimeError: Error(s) in loading state_dict for QuantModel:
	Missing key(s) in state_dict: "model.time_embed.0.weight_quantizer.zero_point", "model.time_embed.0.weight_quantizer.delta", "model.time_embed.2.weight_quantizer.zero_point", "model.time_embed.2.weight_quantizer.delta", "model.input_blocks.0.0.weight_quantizer.zero_point", "model.input_blocks.0.0.weight_quantizer.delta", "model.input_blocks.1.0.in_layers.2.weight_quantizer.zero_point", "model.input_blocks.1.0.in_layers.2.weight_quantizer.delta", "model.input_blocks.1.0.emb_layers.1.weight_quantizer.zero_point", "model.input_blocks.1.0.emb_layers.1.weight_quantizer.delta", "model.input_blocks.1.0.out_layers.3.weight_quantizer.zero_point", "model.input_blocks.1.0.out_layers.3.weight_quantizer.delta", "model.input_blocks.1.1.proj_in.weight_quantizer.zero_point", "model.input_blocks.1.1.proj_in.weight_quantizer.delta", "model.input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight_quantizer.zero_point", "model.input_blocks.1.1.t

In [14]:
#sampler.model.model.diffusion_model = qnn

In [21]:
type(qnn.model)

ldm.modules.diffusionmodules.openaimodel.UNetModel

In [22]:
unet = model.model.diffusion_model

In [39]:
qunet = qnn.model

In [40]:
len(qunet.input_blocks),len(qunet.middle_block),len(qunet.output_blocks)

(12, 3, 12)

In [41]:
lt = lambda l : str(type(l)).split('.')[-1].split('\'')[0]

In [42]:
in_b =[]
for i,inb_i in enumerate(qunet.input_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['QuantModule'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['QuantResBlock'],
 ['QuantResBlock']]

In [43]:
in_b=[]
for l in qunet.middle_block:
    in_b.append(lt(l))
in_b

['QuantResBlock', 'SpatialTransformer', 'QuantResBlock']

In [44]:
in_b =[]
for i,inb_i in enumerate(qunet.output_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['QuantResBlock'],
 ['QuantResBlock'],
 ['QuantResBlock', 'Upsample'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer', 'Upsample'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer', 'Upsample'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer'],
 ['QuantResBlock', 'SpatialTransformer']]

In [45]:
c2d = qunet.input_blocks[0][0]
rb = qunet.input_blocks[1][0]
us = qunet.output_blocks[2][1]
ds = qunet.input_blocks[3][0]
st = qunet.input_blocks[1][1]
rb_u = qunet.output_blocks[0][0]
type(c2d),type(rb),type(ds),type(us),type(st),type(rb_u)

(qdiff.quant_layer.QuantModule,
 qdiff.quant_block.QuantResBlock,
 ldm.modules.diffusionmodules.openaimodel.Downsample,
 ldm.modules.diffusionmodules.openaimodel.Upsample,
 ldm.modules.attention.SpatialTransformer,
 qdiff.quant_block.QuantResBlock)

In [46]:
rb.full_name,type(rb)

('model.input_blocks.1.0', qdiff.quant_block.QuantResBlock)

In [47]:
rb.in_layers

Sequential(
  (0): QuantOp(
    (fwd_func): GroupNorm32(32, 320, eps=1e-05, affine=True)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
  (1): QuantOp(
    (fwd_func): SiLU()
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
  (2): QuantModule(
    320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (weight_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=True, channel_wise=True, leaf_param=False)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [48]:
rb.emb_layers

Sequential(
  (0): QuantOp(
    (fwd_func): SiLU()
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
  (1): QuantModule(
    in_features=1280, out_features=320, bias=True
    (weight_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=True, channel_wise=True, leaf_param=False)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [25]:
#qunet.output_blocks[0][0].in_layers

In [30]:
rb.updown

False

In [31]:
rb.in_layers

Sequential(
  (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
  (1): SiLU()
  (2): QuantModule(
    320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (weight_quantizer): AdaRoundQuantizer(bit=8, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [32]:
rb.in_layers[2].activation_function

StraightThrough()

In [33]:
rb.emb_layers

Sequential(
  (0): SiLU()
  (1): QuantModule(
    in_features=1280, out_features=320, bias=True
    (weight_quantizer): AdaRoundQuantizer(bit=8, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [34]:
rb.out_layers

Sequential(
  (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
  (1): SiLU()
  (2): Dropout(p=0, inplace=False)
  (3): QuantModule(
    320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (weight_quantizer): AdaRoundQuantizer(bit=8, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [35]:
ds

Downsample(
  (op): QuantModule(
    320, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
    (weight_quantizer): AdaRoundQuantizer(bit=8, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
)

In [36]:
type(st)

ldm.modules.attention.SpatialTransformer

In [37]:
st.norm

GroupNorm(32, 320, eps=1e-06, affine=True)

In [34]:
st.proj_in

QuantModule(
  320, 320, kernel_size=(1, 1), stride=(1, 1)
  (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
  (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
  (activation_function): StraightThrough()
)

In [36]:
st.transformer_blocks[0].attn1
#.qk_matmul#.qk_matmul

CrossAttention(
  (to_q): QuantModule(
    in_features=320, out_features=320, bias=False
    (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
  (to_k): QuantModule(
    in_features=320, out_features=320, bias=False
    (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
    (activation_function): StraightThrough()
  )
  (to_v): QuantModule(
    in_features=320, out_features=320, bias=False
    (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
    (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
 

In [31]:
rb_u.skip_connection#.updown

QuantModule(
  2560, 1280, kernel_size=(1, 1), stride=(1, 1)
  (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
  (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
  (activation_function): StraightThrough()
  (weight_quantizer_0): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
  (act_quantizer_0): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
)

x.shape = torch.Size([3, 4, 64, 64])

t.shape = torch.Size([3]) 

c.shape = torch.Size([3, 77, 768])

In [47]:
qnn.model

UNetModel(
  (time_embed): Sequential(
    (0): QuantModule(
      in_features=320, out_features=1280, bias=True
      (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
      (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
      (activation_function): StraightThrough()
    )
    (1): SiLU()
    (2): QuantModule(
      in_features=1280, out_features=1280, bias=True
      (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode=learned_hard_sigmoid)
      (act_quantizer): UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)
      (activation_function): StraightThrough()
    )
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): QuantModule(
        4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (weight_quantizer): AdaRoundQuantizer(bit=4, symmetric=False, round_mode

In [32]:
_=qnn.cuda()
_=qnn.model.cuda()

In [49]:
input_shapes = [[3, 4, 64, 64],[3],[3,77,768]]

In [55]:
import torch
device='cuda'#'cpu'
input_data = [torch.randn(input_shape).to(device) for input_shape in input_shapes]
[input_data[0].shape,input_data[1].shape,input_data[2].shape]
#input_data


[torch.Size([3, 4, 64, 64]), torch.Size([3]), torch.Size([3, 77, 768])]

In [51]:
from ldm.modules.diffusionmodules.openaimodel import ResBlock
from qdiff.quant_block import QuantResBlock

In [43]:
qnn.set_quant_state(True, False)

In [ ]:
for name,module in qnn.named_modules():
    if isinstance(module,QuantModule,Quant):
        print(name,module.full_name)

In [ ]:
#cali_data = (torch.randn(1, 4, 64, 64), torch.randint(0, 1000, (1,)), torch.randn(1, 77, 768))
#resume_cali_model(qnn, opt.cali_ckpt, cali_data, opt.quant_act, "qdiff", cond=opt.cond)


In [61]:
from ldm.modules.diffusionmodules.util import timestep_embedding
t_emb = timestep_embedding(input_data[1],qnn.model.model_channels, repeat_only=False)
emb = qnn.model.time_embed(t_emb)

In [69]:
in_rs1.shape

torch.Size([3, 320, 64, 64])

In [62]:
in_rs1 = c2d(input_data[0])

In [64]:
len(in_rs1),in_rs1.shape,type(c2d)

(3, torch.Size([3, 320, 64, 64]), qdiff.quant_layer.QuantModule)

In [66]:
rb.skip_connection

Identity()

In [70]:
rb(in_rs1,emb)#.shape

tensor([[[[-6.0483e-01,  9.5565e-02,  6.0763e-02,  ..., -7.1485e-02,
           -7.0646e-01,  3.4055e-01],
          [ 6.4123e-01, -6.7859e-01, -3.0940e-01,  ...,  7.4715e-02,
           -3.9171e-01, -1.1731e+00],
          [-3.2907e-01,  4.4286e-01, -1.7953e-01,  ..., -7.3238e-02,
           -1.1173e-01, -4.3877e-01],
          ...,
          [ 1.4083e-02, -1.8868e-01,  1.3839e-01,  ..., -3.5201e-02,
            4.0740e-01, -2.2696e-01],
          [ 5.6219e-01, -1.6915e-01, -6.0696e-01,  ..., -9.1107e-02,
           -1.1927e+00, -7.2790e-01],
          [-7.0619e-02, -9.9699e-01, -1.4520e-01,  ...,  1.1096e-02,
            8.2167e-01, -8.1911e-01]],

         [[ 4.9296e-01, -3.2073e-01,  6.9494e-01,  ...,  8.4651e-01,
            2.9989e-01, -7.8703e-02],
          [-2.8607e-01, -1.6307e+00,  2.3937e+00,  ..., -1.0739e+00,
           -1.8014e-01,  1.8507e+00],
          [ 3.3252e-01,  2.1991e-01,  9.9357e-01,  ...,  2.1244e-01,
           -6.8687e-02,  2.4210e-01],
          ...,
     

In [76]:
rb.full_name,type(rb)

('model.input_blocks.1.0', qdiff.quant_block.QuantResBlock)

In [77]:
isinstance(rb,BaseQuantBlock)

True

In [86]:
for name, module in rb.named_modules():
    if isinstance(module, (QuantModel,QuantOp)):
        print( module.full_name,module.act_quantizer.leaf_param)

model.input_blocks.1.0.in_layers.0 True
model.input_blocks.1.0.in_layers.1 True
model.input_blocks.1.0.emb_layers.0 True
model.input_blocks.1.0.out_layers.0 True
model.input_blocks.1.0.out_layers.1 True


In [79]:
rb.act_quantizer

UniformAffineQuantizer(bit=8, scale_method=max, symmetric=False, channel_wise=False, leaf_param=True)

In [78]:
hasattr(rb.act_quantizer, 'delta')

True

In [73]:
rb.use_checkpoint = False

In [74]:
rb.eval()
rb.requires_grad_(False)
rb.use_checkpoint = False
traced_resblock = torch.jit.trace(rb,(in_rs1,emb),_store_inputs=True)
traced_resblock.save('traced_Qresblock_Qop.pt')

In [7]:
type(model.model.diffusion_model)

ldm.modules.diffusionmodules.openaimodel.UNetModel

In [8]:
unet = model.model.diffusion_model

In [12]:
len(unet.input_blocks),len(unet.middle_block),len(unet.output_blocks)

(12, 3, 12)

In [45]:
lt = lambda l : str(type(l)).split('.')[-1].split('\'')[0]

In [48]:
a =[] 
a.append([])

In [50]:
a[0]

[]

In [52]:
in_b =[]
for i,inb_i in enumerate(unet.input_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['Conv2d'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock'],
 ['ResBlock']]

In [56]:
in_b=[]
for l in unet.middle_block:
    in_b.append(lt(l))
in_b

['ResBlock', 'SpatialTransformer', 'ResBlock']

In [54]:
in_b =[]
for i,inb_i in enumerate(unet.output_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['ResBlock'],
 ['ResBlock'],
 ['ResBlock', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer']]

In [68]:
type(unet.output_blocks[1][0])

ldm.modules.diffusionmodules.openaimodel.ResBlock

In [76]:
unet.output_blocks[5][0].updown

False

In [152]:
c2d = unet.input_blocks[0][0]
rb = unet.input_blocks[1][0]
us = unet.output_blocks[2][1]
ds = unet.input_blocks[3][0]
st = unet.input_blocks[1][1]
type(c2d),type(rb),type(ds),type(us),type(st)

(torch.nn.modules.conv.Conv2d,
 ldm.modules.diffusionmodules.openaimodel.ResBlock,
 ldm.modules.diffusionmodules.openaimodel.Downsample,
 ldm.modules.diffusionmodules.openaimodel.Upsample,
 ldm.modules.attention.SpatialTransformer)

In [153]:
ds.use_conv,us.use_conv

(True, True)

In [154]:
rb.in_layers

Sequential(
  (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
  (1): SiLU()
  (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [155]:
rb.emb_layers

Sequential(
  (0): SiLU()
  (1): Linear(in_features=1280, out_features=320, bias=True)
)

In [156]:
rb.out_layers

Sequential(
  (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
  (1): SiLU()
  (2): Dropout(p=0, inplace=False)
  (3): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [84]:
rb.skip_connection

Identity()

In [162]:
st.norm,st.proj_in,st.proj_out

(GroupNorm(32, 320, eps=1e-06, affine=True),
 Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1)),
 Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1)))

In [161]:
len(st.transformer_blocks)

1

In [165]:
st.transformer_blocks[0].attn1, st.transformer_blocks[0].attn2

(CrossAttention(
   (to_q): Linear(in_features=320, out_features=320, bias=False)
   (to_k): Linear(in_features=320, out_features=320, bias=False)
   (to_v): Linear(in_features=320, out_features=320, bias=False)
   (qk_matmul): CrossQKMatMul()
   (smv_matmul): CrossSMVMatMul()
   (to_out): Sequential(
     (0): Linear(in_features=320, out_features=320, bias=True)
     (1): Dropout(p=0.0, inplace=False)
   )
 ),
 CrossAttention(
   (to_q): Linear(in_features=320, out_features=320, bias=False)
   (to_k): Linear(in_features=768, out_features=320, bias=False)
   (to_v): Linear(in_features=768, out_features=320, bias=False)
   (qk_matmul): CrossQKMatMul()
   (smv_matmul): CrossSMVMatMul()
   (to_out): Sequential(
     (0): Linear(in_features=320, out_features=320, bias=True)
     (1): Dropout(p=0.0, inplace=False)
   )
 ))

In [41]:
inb_i = unet.input_blocks[0]
#len(inb_i)
for l in inb_i:
    print(type(l))

<class 'torch.nn.modules.conv.Conv2d'>


In [46]:
lt(l)

'Conv2d'

In [ ]:
rb

In [51]:
model.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3, 

In [85]:
import torch
device='cpu'
input_data = [torch.randn(input_shape).to(device) for input_shape in input_shapes]
[input_data[0].shape,input_data[1].shape,input_data[2].shape]
#input_data


[torch.Size([3, 4, 64, 64]), torch.Size([3]), torch.Size([3, 77, 768])]

In [86]:
input_data = tuple(input_data)

In [88]:
input_data[2].device

device(type='cpu')

In [87]:
_=model.cpu()

In [55]:
input_unet = {'x':input_data[0],'timesteps':input_data[1],'context':input_data[2]}

In [105]:
from torchviz import make_dot

In [92]:
from ldm.modules.diffusionmodules.util import timestep_embedding
t_emb = timestep_embedding(input_data[1], model.model.diffusion_model.model_channels, repeat_only=False)
emb = model.model.diffusion_model.time_embed(t_emb)

In [93]:
emb.shape

torch.Size([3, 1280])

In [101]:
in_rs1 = c2d(input_data[0])

In [110]:
rs1_out= rb._forward(in_rs1,emb)

In [111]:
#make_dot(rs1_out, params=dict(list(rb.named_parameters()))).render("rb1", format="svg")
make_dot(rs1_out, params=None).render("rb1", format="svg")



'rb1.svg'

In [137]:
_=rb.eval()

In [141]:
traced_resblock = torch.jit.script(rb)#._forward,example_inputs=[rb,in_rs1,emb])
#traced_resblock.save('script_resblock.pt')

RuntimeError: Can't redefine method: forward on class: __torch__.ldm.modules.diffusionmodules.util.___torch_mangle_186.GroupNorm32 (of Python compilation unit at: 0x5594424b26a0)

In [142]:
rb.eval()
rb.requires_grad_(False)
traced_resblock = torch.jit.trace_module(rb,{'_forward':(in_rs1,emb)},_store_inputs=True)
traced_resblock.save('traced_resblock.pt')

/opt/conda/envs/qdiff_reorg/lib/python3.8/site-packages/torch/jit/_trace.py:168: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789116784/work/build/aten/src/ATen/core/TensorBody.h:489.)
  if a.grad is not None:


In [ ]:
with torch.no_grad():
    torch.onnx.export(rb._forward,(in_rs1,emb),'resblock.onnx',
                   input_names=['x','timestep_embedding'], output_names=['res_block_out'])

AttributeError: 'function' object has no attribute 'modules'

In [90]:
model.model.diffusion_model.model_channels

320

In [88]:
yhat = model.model.diffusion_model(*input_data)

In [96]:
make_dot(yhat, params=dict(list(model.model.diffusion_model.named_parameters()))).render("rnn_torchviz", format="png")


'rnn_torchviz.png'

In [89]:
model.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3, 

In [83]:
import hiddenlayer as hl

transforms = [ hl.transforms.Prune('Constant') ] #

In [91]:
graph = hl.build_graph(model.model.diffusion_model, input_data, transforms=transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('rnn_hiddenlayer', format='png')

IndexError: Argument passed to at() was not in the map.

In [56]:
#outout_unet= model.model.diffusion_model(**input_unet)
outout_unet= model.model.diffusion_model(*input_data)

outout_unet.shape

torch.Size([3, 4, 64, 64])

In [79]:
input_names = ['x','timesteps','context']
output_names = ['h']
torch.onnx.export(model.model.diffusion_model,(input_data[0],input_data[1],input_data[2]),
                   'UNetModel.onnx',
                   input_names=input_names, output_names=output_names)

RuntimeError: _Map_base::at

In [90]:
torch.onnx._optimize_trace

AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'

In [75]:
traced_model = torch.jit.trace(model.model.diffusion_model, (input_data[0],input_data[1],input_data[2]))
#traced_model = torch.jit.trace(model.model.diffusion_model, input_data)

#traced_model = torch.jit.trace(model.model.diffusion_model, **input_unet)



RuntimeError: _Map_base::at

In [14]:
create_torch_script(model.model.diffusion_model,input_shapes)

RuntimeError: _Map_base::at